In [1]:
# Import the necessary libraries
import os
import re
import time
import json
import psutil
import numpy as np
import pandas as pd
import concurrent.futures
from io import StringIO
from tqdm import tqdm
import concurrent.futures
import psutil
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

import warnings 
warnings.filterwarnings("ignore")

# Single Thread Code

In [2]:

# Remove progress file if it exists
if os.path.exists('progress.json'):
    os.remove('progress.json')

# Initialize webdriver with implicit waits
options = Options()
options.add_argument("--headless")  # Uncomment this line to run the browser in headless mode
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.implicitly_wait(10)  # Set the implicit wait to 10 seconds

# Define a function to save progress
def save_progress(category, page):
    with open('progress.json', 'w') as f:
        json.dump({'category': category, 'page': page}, f)

# Define a function to load progress
def load_progress():
    try:
        with open('progress.json', 'r') as f:
            progress = json.load(f)
            return progress['category'], progress['page']
    except FileNotFoundError:
        return None, None

# Load progress
last_processed_category, last_processed_page = load_progress()

start_processing = False
if not last_processed_category:
    start_processing = True


# Print CPU usage statistics
cpu_usage = psutil.cpu_percent(interval=1)
print(f'CPU usage: {cpu_usage}%')

# Define categories and number of pages for each category
categories = [
    'sut-kahvaltilik',
    'temel-gida',
    'unlu-mamul-pasta',
    'icecek',
    'atistirmalik',
    'meze-hazir-yemek-donuk',
    'bebek-anne',
    'deterjan-temizlik'
]

pages = [
    25,  # Number of pages for 'sut-kahvaltilik'
    29,  # Number of pages for 'temel-gida'
    9,   # Number of pages for 'unlu-mamul-pasta'
    22,  # Number of pages for 'icecek'
    16,  # Number of pages for 'atistirmalik'
    2,   # Number of pages for 'meze-hazir-yemek-donuk'
    13,  # Number of pages for 'bebek-anne'
    29   # Number of pages for 'deterjan-temizlik'
]

# Define additional URLs for each category
extra_pages = {
    'bebek-anne': [
        "https://marketkarsilastir.com/ara/kotex"
    ], 
    'temel-gida': [
        "https://marketkarsilastir.com/ara/oncu"
    ],
    'sut-kahvaltilik': [
        "https://marketkarsilastir.com/ara/yorukoglu"
    ],
    'atistirmalik': [
        "https://marketkarsilastir.com/ara/lays"
    ],
    'deterjan-temizlik': [
        "https://marketkarsilastir.com/ara/abc",
        "https://marketkarsilastir.com/ara/vanish",
        "https://marketkarsilastir.com/ara/ariel",
        "https://marketkarsilastir.com/ara/bref",
        "https://marketkarsilastir.com/ara/clin",
        "https://marketkarsilastir.com/ara/clear",
        "https://marketkarsilastir.com/ara/head-and-shoulders",
        "https://marketkarsilastir.com/ara/gliss",
        "https://marketkarsilastir.com/ara/garnier",
        "https://marketkarsilastir.com/ara/loreal",
        "https://marketkarsilastir.com/ara/protex",
        "https://marketkarsilastir.com/ara/duru",
        "https://marketkarsilastir.com/ara/ipek",
        "https://marketkarsilastir.com/ara/nivea",
        "https://marketkarsilastir.com/ara/peros",
        "https://marketkarsilastir.com/ara/rexona",
        "https://marketkarsilastir.com/ara/selpak",
        "https://marketkarsilastir.com/ara/solo",
        "https://marketkarsilastir.com/ara/papia",
        "https://marketkarsilastir.com/ara/calgon",
        "https://marketkarsilastir.com/ara/colgate",
        "https://marketkarsilastir.com/ara/pantene"
    ]
    # Add additional categories and URLs as needed...
}

df_list = []

# Your data extraction function
def extract_data(category, url, page):
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'lxml')

    # Get all product links
    product_links = [a['href'] for a in soup.select('a.product-img')]

    # Loop over each product link
    for link in product_links:
        product_url = 'https://marketkarsilastir.com' + ('' if link.startswith('/') else '/') + link
        driver.get(product_url)

        product_soup = BeautifulSoup(driver.page_source, 'lxml')

        # Extract details
        try:
            product_name_tag = product_soup.select_one('h1.text-center')
            product_name = product_name_tag.text.strip() if product_name_tag else 'No name available'

            product_barcode_tag = product_soup.select_one('h2.fa-1x.text-center')
            product_barcode = product_barcode_tag.text.strip() if product_barcode_tag else 'No barcode available'

            product_picture_link_tag = product_soup.select_one('img.img-responsive')
            product_picture_link = product_picture_link_tag['src'] if product_picture_link_tag else 'No image available'

            # Extract product info
            product_info_section = product_soup.find('div', class_='barkodResim')
            product_info = product_info_section.text.strip() if product_info_section else 'No product info available'

            # Extract specific info fields
            net_miktar = 'N/A'
            mensi = 'N/A'
            icindekiler = 'N/A'
            alerjen_uyarisi = 'N/A'
            saklama_kosullari = 'N/A'
            gida_isletmecisi = 'N/A'

            info_items = product_info_section.find_all('p')
            for item in info_items:
                strong_tag = item.find('strong')
                if strong_tag:
                    strong_text = strong_tag.text.strip()
                    if strong_text == 'Net Miktar (g/ml)':
                        net_miktar = item.contents[-1].strip()
                    elif strong_text == 'Menşei':
                        mensi = item.contents[-1].strip()
                    elif strong_text == 'İçindekiler':
                        icindekiler = item.contents[-1].strip()
                    elif strong_text == 'Alerjen Uyarısı':
                        alerjen_uyarisi = item.contents[-1].strip()
                    elif strong_text == 'Saklama Koşulları':
                        saklama_kosullari = item.contents[-1].strip()
                    elif strong_text == 'Gıda İşletmecisi / Üretici / İthalatçı / Dağıtıcı':
                        gida_isletmecisi = item.contents[-1].strip()

            # Extract product price info
            product_price_info_section = product_soup.find('div', class_='col-xl-8')
            product_price_info_rows = product_price_info_section.select('tbody tr')

            for row in product_price_info_rows:
                market_name = row.select_one('td a img')['title']
                sku_name = row.select_one('td:nth-child(3) a').text.strip()
                sku_price = row.select_one('td:nth-child(4)').text.strip()
                latest_price_change = row.select_one('td:nth-child(3) small').text.strip()

                # Save details to dataframe
                df_list.append({
                    'TARIX': datetime.today(),
                    'KATEQORIYA': category,
                    'MEHSUL AD': product_name,
                    'MEHSUL BARKOD': product_barcode,
                    'NET MIKTAR': net_miktar,
                    'MENSEİ': mensi,
                    'İÇİNDEKİLER': icindekiler,
                    'ALERJEN UYARISI': alerjen_uyarisi,
                    'SAKLAMA KOŞULLARI': saklama_kosullari,
                    'GIDA İŞLETMECİSİ / ÜRETİCİ / İTHALATÇI / DAĞITICI': gida_isletmecisi,
                    'MARKET ADI': market_name,
                    'SKU AD': sku_name,
                    'SKU QIYMET': sku_price,
                    'SON QIYMET DEYİŞİM TARİXİ': latest_price_change,
                    'MEHSUL SEKİL LİNKİ': product_picture_link,
                    'MEHSUL LİNKI': product_url
                })

                # Save the DataFrame after each item
                df = pd.DataFrame(df_list)
                df.to_csv(filename)
                save_progress(category, page)

        except Exception as e:
            print(f"Error extracting data from {product_url}: {str(e)}")

# Get the current date as a string
date_string = datetime.now().strftime('%Y-%m-%d')

# Create the filename
filename = '/Users/FaiqRasulov/Desktop/Qiymet_list/TOPDANLAR/Market_Karshilashtir/marketkarsilastir_{date_string}.csv'

# Start extracting data from the original URLs
for category, page_count in zip(categories, pages):
    if category == last_processed_category:
        start_processing = True
    if start_processing:
        for page in range(last_processed_page if category == last_processed_category else 1, page_count + 1):
            try:
                url = f"https://marketkarsilastir.com/urunkat/{category}/?sayfa={page}"
                extract_data(category, url, page)
            except TimeoutException as e:
                print(f"TimeoutException occurred: {str(e)}. The data scraped so far has been saved.")
                break

# Now extract data from the additional URLs
for category, urls in extra_pages.items():
    for url in urls:
        try:
            extract_data(category, url, None)

            # Save the DataFrame after each page
            df = pd.DataFrame(df_list)
            df.to_csv(filename)
        except TimeoutException as e:
            print(f"TimeoutException occurred: {str(e)}. The data scraped so far has been saved.")
            break

# Close the driver
driver.quit()

# Create DataFrame from the list of dictionaries
df = pd.DataFrame(df_list)

# Rearrange the columns
df = df[['TARIX', 'KATEQORIYA', 'MEHSUL AD', 'MEHSUL BARKOD', 'NET MIKTAR', 'MENSEİ', 'İÇİNDEKİLER',
         'ALERJEN UYARISI', 'SAKLAMA KOŞULLARI', 'GIDA İŞLETMECİSİ / ÜRETİCİ / İTHALATÇI / DAĞITICI',
         'MARKET ADI', 'SKU AD', 'SKU QIYMET', 'SON QIYMET DEYİŞİM TARİXİ', 'MEHSUL SEKİL LİNKİ', 'MEHSUL LİNKI']]

# Save to CSV
df.to_csv(filename, index=False)

print("Data extraction completed successfully!")


[WDM] - Downloading: 100%|██████████| 8.79M/8.79M [00:02<00:00, 3.59MB/s]


CPU usage: 47.9%
Error extracting data from https://marketkarsilastir.com/fiyat/3161911393210-elle-vire-tuzlu-tereyagi-200-g: 'NoneType' object is not subscriptable
Error extracting data from https://marketkarsilastir.com/fiyat/4000521642006-dr-oetker-vitalis-meyveli-musli-375-g: 'NoneType' object is not subscriptable
Error extracting data from https://marketkarsilastir.com/fiyat/5900020008930-nestle-musli-uzumlu-findikli-350-gr: 'NoneType' object is not subscriptable
Error extracting data from https://marketkarsilastir.com/fiyat/8000500082379-nutella-1000-g: 'NoneType' object is not subscriptable
Error extracting data from https://marketkarsilastir.com/fiyat/8000500279274-nutella-minyonlar-825-g: 'NoneType' object is not subscriptable
Error extracting data from https://marketkarsilastir.com/fiyat/8002795002469-sterilgarda-uht-tam-yagli-mascarpone-200-g: 'NoneType' object is not subscriptable
Error extracting data from https://marketkarsilastir.com/fiyat/8004673600194-latteria-soresina

In [3]:
df['KATEQORIYA'] = df['KATEQORIYA'].str.upper()

In [4]:
links_to_delete = [
    "https://marketkarsilastir.com/fiyat/8697353501982-ozdilek-mickey-minnie-mouse-u-and-me-tek-kisilik-disney-lisansli-cocuk-pike-takimi",
    "https://marketkarsilastir.com/fiyat/8681174580592-slazenger-erkek-mont-head-lacivert",
    "https://marketkarsilastir.com/fiyat/8698489640354-kom-erkek-slip-ariel-44sl60021", 
    "https://marketkarsilastir.com/fiyat/8698532591596-ped-tasima-kabi"
]

df = df[~df["MEHSUL LİNKI"].isin(links_to_delete)].reset_index(drop=True)


In [5]:
# Initial number of rows
initial_rows = df.shape[0]
print(f"Initial number of rows: {initial_rows}")

# Remove duplicates
df = df.drop_duplicates()

# Final number of rows
final_rows = df.shape[0]
print(f"Final number of rows: {final_rows}")

# Display how many duplicates were removed
print(f"Number of duplicates removed: {initial_rows - final_rows}")

# Reset the index to get the row numbers
df = df.reset_index(drop=True)

Initial number of rows: 26971
Final number of rows: 26971
Number of duplicates removed: 0


In [6]:
df1 = df.copy()# Create a copy of the dataframe
df1

,TARIX,KATEQORIYA,MEHSUL AD,MEHSUL BARKOD,NET MIKTAR,MENSEİ,İÇİNDEKİLER,ALERJEN UYARISI,SAKLAMA KOŞULLARI,GIDA İŞLETMECİSİ / ÜRETİCİ / İTHALATÇI / DAĞITICI,MARKET ADI,SKU AD,SKU QIYMET,SON QIYMET DEYİŞİM TARİXİ,MEHSUL SEKİL LİNKİ,MEHSUL LİNKI
0,2023-06-16 10:10:08.305330,SUT-KAHVALTILIK,Nutella 400 G Market\nFiyatları,Barkod Numarası : 80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,Bizim Toptan,Nutella 400 G Son Fiyat Değişim Tarihi:\n(24.05.2023 14:33),"53,90",Son Fiyat Değişim Tarihi:\n(24.05.2023 14:33),https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g
1,2023-06-16 10:10:08.327225,SUT-KAHVALTILIK,Nutella 400 G Market\nFiyatları,Barkod Numarası : 80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,Özdilek Hipermarketleri,Nutella 400 G Son Fiyat Değişim Tarihi:\n(09.05.2023 01:26),"54,95",Son Fiyat Değişim Tarihi:\n(09.05.2023 01:26),https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g
2,2023-06-16 10:10:08.332513,SUT-KAHVALTILIK,Nutella 400 G Market\nFiyatları,Barkod Numarası : 80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,CarrefourSA,Nutella 400 G Son Fiyat Değişim Tarihi:\n(03.05.2023 01:29),"55,00",Son Fiyat Değişim Tarihi:\n(03.05.2023 01:29),https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g
3,2023-06-16 10:10:08.336578,SUT-KAHVALTILIK,Nutella 400 G Market\nFiyatları,Barkod Numarası : 80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,Macrocenter,Nutella 400 G Son Fiyat Değişim Tarihi:\n(05.05.2023 04:34),"55,00",Son Fiyat Değişim Tarihi:\n(05.05.2023 04:34),https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g
4,2023-06-16 10:10:08.341713,SUT-KAHVALTILIK,Nutella 400 G Market\nFiyatları,Barkod Numarası : 80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,İstegelsin,Nutella 400 G Son Fiyat Değişim Tarihi:\n(08.05.2023 11:27),"55,00",Son Fiyat Değişim Tarihi:\n(08.05.2023 11:27),https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26966,2023-06-16 15:36:18.527558,DETERJAN-TEMIZLIK,Pantene Saç Kremi Yag Terapisi 275ML Market\nFiyatları,Barkod Numarası : 8006540440728,N/A,N/A,N/A,N/A,N/A,N/A,Metro Toptancı Market,Pantene Saç Kremi Yag Terapisi 275ML Son Fiyat Değişim Tarihi:\n(12.01.2023 09:37),"45,90",Son Fiyat Değişim Tarihi:\n(12.01.2023 09:37),https://marketkarsilastir.com/urunler/8006540440728.jpg,https://marketk

In [7]:
df1['MEHSUL AD'] = df1['MEHSUL AD'].str.replace(' Market Fiyatları', '').str.replace('\n', '').str.strip()
df1['MEHSUL AD'] = df1['MEHSUL AD'].str.replace('MarketFiyatları', '').str.strip()
df1['MEHSUL BARKOD'] = df1['MEHSUL BARKOD'].astype(str).str.replace('Barkod Numarası : ', '').astype(float).round(0)
df1['SKU AD'] = df1['SKU AD'].str.split(' Son Fiyat Değişim Tarihi').str[0]
df1['SKU QIYMET'] = df1['SKU QIYMET'].astype(str).str.replace('.', '').str.replace(',', '.').astype(float).round(2)

# Remove the unwanted strings and any whitespace:
df1['SON QIYMET DEYİŞİM TARİXİ'] = df1['SON QIYMET DEYİŞİM TARİXİ'].str.replace('Son Fiyat Değişim Tarihi:', '')
df1['SON QIYMET DEYİŞİM TARİXİ'] = df1['SON QIYMET DEYİŞİM TARİXİ'].str.replace('\(', '')
df1['SON QIYMET DEYİŞİM TARİXİ'] = df1['SON QIYMET DEYİŞİM TARİXİ'].str.replace('\)', '')
df1['SON QIYMET DEYİŞİM TARİXİ'] = df1['SON QIYMET DEYİŞİM TARİXİ'].str.replace('\n', '')
df1['SON QIYMET DEYİŞİM TARİXİ'] = pd.to_datetime(df1['SON QIYMET DEYİŞİM TARİXİ'], format='%d.%m.%Y %H:%M')


# Convert 'NET MIKTAR' column to integers
df1['NET MIKTAR'] = pd.to_numeric(df1['NET MIKTAR'], errors='coerce').astype(pd.Int64Dtype())

# Convert the MEHSUL BARKOD column to integer and then back to string
df1['MEHSUL BARKOD'] = df1['MEHSUL BARKOD'].astype(float).astype(int).astype(str)


df1['TARIX'] = pd.to_datetime(df1['TARIX'])


In [8]:
brands = [
    'Nutella', 'Pınar', 'Bahçıvan', 'La Vache', 'Kiri', 'Elle Vire', 'Alpro', 'Blue Heaven', 'Pinafore', 'Lactima',
    'Nesfit', 'Nestle', 'Castelli', 'Torku', 'Grana Padano', 'Frico', 'Hünnap', 'Sütaş', 'Beeo', 'Laz Kızı',
    'Tarış', 'Tarıs', 'Marmarabirlik', 'Sek', 'Tkv', 'Ülker', 'İçim', 'Tukaş', 'Eti', 'Sarelle', 'Tamek',
    'Nestlé', 'Nesquik', 'Sana', 'Magnum', 'Max', 'Algida', 'Cornetto', 'Becel', 'Calve', 'Koska', 'Yörükoğlu',
    'Balparmak', 'Gold', 'Fo', 'Yörsan', 'Tahsildaroğlu', 'Eker', 'Bizim', 'Penguen', 'Trakya', 'G.c.',
    'Ricotta', 'Fiskobirlik', 'Fiskokrem', 'Altınkılıç', 'Kebır Köy', 'Alpella', 'President', 'Zertum', 'Aynes',
    'Muratbey', 'Anavarza', 'Activia', 'Danone', 'Activa', 'Danette', 'Tikveşli', 'Doğalsan', 'Ekici', 'Cem',
    'Duranlar', 'Cebel', 'Gündoğdu', 'Sinangil', 'Milupa', 'Şitoğlu', 'Tire Süt', 'Baltalı', 'Fora',
    'St. Dalfour', 'Tat', 'Danino', 'Karper', 'Kebir', 'Kühne', 'Quaker', 'Tariş', 'Veggy', 'Ayca', 'Buzlat',
    'Doğruluk', 'Santral', 'Selva', 'Taciroğlu', 'Berrak', 'Yenigün', 'Dimes', 'Teksüt', 'Kelloggs',
    'Helvart', 'Mornflake', 'Soma', 'Fabrikita', 'City Farm', 'Wefood', 'Ariste', 'Vegan', 'Orfa The Standart Vegan',
    'Hasmandıra', 'Bonus', 'Friendship', 'Polonezköy', 'Nesquık', 'Coşkun', 'Bigjoy', 'Sütbon', 'Casari',
    'Akpınar', 'Koop', 'Kirmasti', 'Fomilk', 'Saklı Cennet Organik', 'Vivo', 'Vıvo', 'Carte Dor', 'Viennetta',
    'Züber', 'Zazella', 'Feyz', 'Şahbaz Çaylı', 'Fora', 'Heinz', 'Tabasco', 'Franks', 'Amoy', 'Teptip',
    'Lea & Perrins', 'HP', 'Maldon', 'Tuzot', 'Viru', 'De Cecco', 'Rummo', 'Elle Esse', 'Antıco Nero',
    'Okomesan', 'Barilla', 'Ege', 'Zel', 'Tarım Kredi Birlik', 'Türkiye Tarım Kredi', 'Türk Şeker Tarım Kristal',
    'Biryağ', 'Efsina', 'Makarneks', 'Tukas', 'Dardanel', 'Mister', 'Bağdat', 'Söke', 'Filiz', 'Superfresh',
    'Superton', 'Dr. Oetker', 'Calvé', 'Knorr', 'Pastavilla', 'Doğuş', 'Billur', 'Balküpü', 'Kristal', 'Reis',
    'Yudum', 'Sırma', 'Mutlu', 'Orkide', 'Takita', 'Fide', 'Duru', 'Burcu', 'İpek', 'Eriş', 'Çotanak', 'Doğanay',
    'Yayla', 'Kemal', 'Kırlangıç', 'Arbella', 'Şahin', 'Olca', 'Melis', 'Mezzetürk', 'Komılı', 'Komili',
    'Orvital Organik', 'Orvıtal Organik', 'Purelife', 'Kikkoman', 'Pantai', 'Bünsa', 'Olin', 'Fersan', 'Irmak',
    'Reıs', 'Ferfresh', 'Amora', 'De Nigris', 'Delizia', 'Dolco', 'Gullon', 'Lee Kum', 'Poco Loco', 'Tas',
    'Sriracha', 'Vertmont', 'Demko', 'T.a.s', 'Salat', 'Ta-ze', 'Turna', 'Real Thai', 'Dragon Pearl', 'Kuhne',
    'M Life', 'Zaika', 'Veg & Bones', 'Veg&bones', 'Safir', 'Sevgisun', 'Ots', 'Gökbayrak', 'Sade Organik',
    'Bemtat', 'Zade', 'Suree', 'Maille', 'Larina', 'Del Monte', 'Dardanel', 'Alaeddin Ançüez', 'Sade', 'Tassya',
    'Nora', 'Hellmanns', 'Zeroshot', 'Emek', 'Oruçoğlu', 'Abalı', 'Squid', 'Wasa', 'Balocco', 'Hamurdan',
    'Pasta Giulietta', 'Tazedo', 'Etimek', 'Kenton', 'Dr.Oetker', 'Uno', 'Pakmaya', 'Köksal', 'Kral', 'Arı',
    'İhe', 'Dr Oetker', 'İstanbul Organik Halk Ekmek', 'Arslanoğlu', 'İrem', 'Lavalia', 'Embi', 'Saffet',
    'Kireçburnu', 'Pumpernickel', 'Roggen Hafer Brot', 'Volkorn', 'Sonnenblumenkern', 'Feast', 'Myfresh',
    'Dreikorn', 'Pek Food', 'Unabella', 'Delba', 'Glutensiz', 'Uneller', 'Toblerone', 'Milka', 'Oreo', 'Dr.oetker',
    'Alaçatı', 'Cappy', 'Coca-Cola', 'Uludağ', 'Coca Cola', 'Damla', 'Perrier', 'Capri Sun', 'Nescafe', 'Burn',
    'Fuse Tea', 'Sprite', 'Fanta', 'Jacobs', 'Acqua Panna', 'San Pellegrino', 'İlly', 'Assu', 'Beyoğlu',
    'Kocatepe', 'Çaykur', 'Didi', 'Cafe Crown', 'Cola Turka', 'Çamlıca', 'Maltana', 'Obsesso', 'Fruko', 'Lipton',
    'Pepsi', 'Tropicana', 'Yedigün', 'Kavaklıdere', 'Kurukahveci Mehmet Efendi', 'Vefa', 'Nescafé', 'Starbucks',
    'Lıpton', 'Hisar', 'Frutti', 'Erikli', 'Beypazarı', 'Karali', 'Kızılay', 'Hayat', 'Akmina', 'Tirebolu',
    'Exotic', 'Elite', 'Kahve Dünyası', 'Saka', 'Balaban', 'Abant', 'Doğadan', 'Tchibo', 'Hamidiye', 'Akmına',
    'Serafressh', 'Serfresh', 'Davidoff', 'Brazilian', 'Green Life', 'Dünya', 'Pfanner', 'Pürsu Elite Doğal',
    'Fropie', 'Elmacık', 'Reina', 'Özerhisar', 'Serafresh', 'Vivident', 'Vıvıdent', 'Mentos', 'Big Babol',
    'Vıvdent', 'Chupa Chups', 'Polo', 'After Eight', 'Pringles', 'Seven Days', 'Lotus', 'Olips', 'Kent', 'Jelibon',
    'Bubblıcıous', 'Bubblicious', 'First', 'Fırst', 'Patos Critos', 'Chips Master', 'Mixmey', 'Mıxmey', 'Cipso',
    'Patos', 'Chıps Master', 'Mevlana', 'Probis', 'Hanımeller', 'Altınbaşak', 'Rondo', 'Rulokat', 'Dido', 'Ece',
    'Metro', 'Albeni', 'Halley', 'Coco Star', 'Dankek', 'Kekstra', 'Krispi', 'Godiva', 'Caramio', 'Jelıbon',
    'Falım', 'Cheetos', 'Ruffles', 'Çerezza', 'Rocco', 'Tadım', 'Oneo', 'Yıldız', 'Harıbo', 'Haribo', 'Luppo',
    'Greta', 'Biscolata', 'Şölen', 'Peyman', 'Çitliyo', 'Kd.', 'Elit', 'Lolipop', 'Nutzz', 'Lindt', 'Schogetten',
    'Ritter', 'Ozmo', 'Gol', 'Hoşbeş', 'Uni', 'Johnsons', 'Nivea', 'Nıvea', 'Aptamil', 'Prima', 'Prıma',
    'Pampers', 'Hipp', 'Sebamed', 'Huggies', 'Hero', 'Sleepy', 'Roxy', 'Dalan', 'Molfix', 'Dalin', 'Yumoş', 'Omo',
    'Yumos', 'Bebedor', 'Canbebe', 'Wee Baby', 'Hacı Şakir', 'Sma 2', 'Bepanthol', 'Bebelac', 'Dalın', 'Koroplast',
    'Unı Baby', 'Tommee Tippee', 'Chicco', 'Cif', 'Fairy', 'Pril', 'Perwoll', 'Scotch Brite', 'Aır Wıck',
    'Air Wick', 'Cillit', 'Vileda', 'Frosch', 'Febreze', 'Ariel', 'Alo', 'Vıleda', 'Glade', 'Ace', 'Zeta', 'Pronto',
    'Parex', 'Marc', 'Lysol', 'Boron', 'Gülşah', 'Fakır', 'Activex', 'Domestos', 'Selpak', 'Detan', 'Solo',
    'Bingo', 'Familia', 'Papia', 'Bıngo', 'Evony', 'Ernet', 'Camsil', 'Banat', 'Finish', 'Woolıte', 'Vanish',
    'Cillit Bang', 'Calgon', 'Kosla Halı', 'Fınısh', 'Aırwıck', 'Dixi', 'Bref', 'Rinso', 'Vim', 'S.brite', 'Nazar',
    'Off', 'Raid', 'Mr Muscle', 'Mr. Muscle', 'Porçöz', 'Defans', 'Decobella', 'Peros', 'Asperox', 'Maylo', 'Sera',
    'Clin', 'Persil', 'Clın', 'Persıl', 'Tursilmatik', 'Vernel', 'Tursılmatık', 'Tursil Matik', 'Faıry', 'Uzay',
    'Dr.Beckmann', 'Dr. Beckman', 'Tursil', 'Awıck', 'Fırça Sapı', 'Bion', 'Fakir', 'Jet', 'Vanısh',
    'Cıllıt Bang', 'Cıf', 'Piknik', 'Aırwick', 'Kotex', 'KOTEX','Dardenel','S.brıte', 'Öncü','ÖNCÜ','ONCU','YÖRÜKOĞLU',
    'Lays', 'LAYS', 'Abc', 'ABC', 'VANISH', 'ARİEL', 'BREF', 'CLIN', 'Neutrogena', 'Clear', 'DEL\.AXE', 'Old Spice', 
    'Head& Shoulders', 'Head&shoulders', 'Head&Shoulders', 'Head & Shoulders', 'Taft', 'HEAD & SHOULD',
    'HOBBY', 'Gliss', 'Glıss', 'Schwarzkopf', 'Garnier', 'Garnıer', 'GARNIER', 'Loreal', 'LOréal', 'Loréal', 'Protex', 'PROTEX',
    'DURU', 'Ipek', 'IPEK', 'İPEK', 'NIVEA', 'PEROS', 'Rexona','REXONA', 'SELPAK', 'SOLO', 'PAPIA', 'Colgate', 'COLGATE',
    'PANTENE','Pantene', 'Head Shoulders'
]

def find_brand(string):
    matches = re.findall(r'({})'.format('|'.join(brands)), string)
    return matches

df1['BRAND'] = df1['SKU AD'].apply(lambda x: find_brand(str(x)))

df1['BRAND'] = df1['BRAND'].apply(lambda x: ', '.join(x).replace('[', '').replace(']', '').replace('\'', ''))


In [9]:
#Clean up brand column 

df1['BRAND'].replace(['Activia, Sade', 'Activa'],['Activia', 'Activia' ],inplace=True)
df1['BRAND'].replace(['Aır Wıck', 'Aırwick', 'Aırwıck', 'Awıck'],['Air Wick', 'Air Wick', 'Air Wick', 'Air Wick'],inplace=True)
df1['BRAND'].replace(['Akmına'],['Akmina'],inplace=True)
df1['BRAND'].replace(['Algida, Sade', 'Algida, Oreo', 'Algida, Max'],['Algida', 'Algida', 'Algida'],inplace=True)
df1['BRAND'].replace(['Alo, Fairy'],['Alo'],inplace=True)
df1['BRAND'].replace(['Alpro, Sade'],['Alpro'],inplace=True)
df1['BRAND'].replace(['Anavarza, Arı'],['Anavarza'],inplace=True)
df1['BRAND'].replace(['Arbella, Glutensiz'],['Arbella'],inplace=True)
df1['BRAND'].replace(['Ariel, Febreze', 'ARİEL'],['Ariel', 'Ariel'],inplace=True)
df1['BRAND'].replace(['Arı, Sade'],['Arı'],inplace=True)
df1['BRAND'].replace(['Arslanoğlu, Kemal, Tat'],['Arslanoğlu'],inplace=True)
df1['BRAND'].replace(['Bahçıvan, Grana Padano'],['Bahçıvan'],inplace=True)
df1['BRAND'].replace(['Balparmak Yayla Ve Ova Çiçek Balı 460 G', 'Balparmak, Yayla'],['Balparmak', 'Balparmak'],inplace=True)
df1['BRAND'].replace(['Barilla, Glutensiz', 'Barilla, Sade'],['Barilla', 'Barilla'],inplace=True)
df1['BRAND'].replace(['Bebedor, Damla'],['Bebedor'],inplace=True)
df1['BRAND'].replace(['Bebelac, Gold', 'Bebelac, Gold, Salat'],['Bebelac', 'Bebelac'],inplace=True)
df1['BRAND'].replace(['Beeo, Arı', 'Beeo, Damla'],['Beeo', 'Beeo'],inplace=True)
df1['BRAND'].replace(['Berrak, Salat'],['Berrak'],inplace=True)
df1['BRAND'].replace(['Billur, Piknik'],['Billur'],inplace=True)
df1['BRAND'].replace(['Bingo, Dalın', 'Bingo, Yıldız', 'Bingo, Mutlu', 'Bıngo'],['Bingo', 'Bingo','Bıngo','Bingo'],inplace=True)
df1['BRAND'].replace(['Bizim, Yayla'],['Bizim'],inplace=True)
df1['BRAND'].replace(['Bubblıcıous'],['Bubblicious'],inplace=True)
df1['BRAND'].replace(['Burn, Gold'],['Burn'],inplace=True)
df1['BRAND'].replace(['Cafe Crown, Sade'],['Cafe Crown'],inplace=True)
df1['BRAND'].replace(['Calvé', 'Calvé, Tat'],['Calve', 'Calve'],inplace=True)
df1['BRAND'].replace(['Çamlıca, Sade'],['Çamlıca'],inplace=True)
df1['BRAND'].replace(['Camsil, Camsil'],['Camsil'],inplace=True)
df1['BRAND'].replace(['Canbebe, Max'],['Canbebe'],inplace=True)
df1['BRAND'].replace(['Çaykur, Burcu', 'Çaykur, Filiz'],['Çaykur', 'Çaykur'],inplace=True)
df1['BRAND'].replace(['Cebel, Sade'],['Cebel'],inplace=True)
df1['BRAND'].replace(['Chıps Master'],['Chips Master'],inplace=True)
df1['BRAND'].replace(['Chupa Chups, Lolipop'],['Chupa Chups'],inplace=True)
df1['BRAND'].replace(['Cif, Gücü', 'Cif, Cif, Gücü', 'Cıf, Gücü'],['Cif','Cif','Cif'],inplace=True)
df1['BRAND'].replace(['Cipso, Sade'],['Cipso'],inplace=True)
df1['BRAND'].replace(['Clın'],['Clin'],inplace=True)
df1['BRAND'].replace(['Coca-Cola'],['Coca Cola'],inplace=True)
df1['BRAND'].replace(['Cornetto, Toblerone'],['Cornetto'],inplace=True)
df1['BRAND'].replace(['Dalin, Alo', 'Dalin, Yıldız', 'Dalın'],['Dalin', 'Dalin', 'Dalin'],inplace=True)
df1['BRAND'].replace(['Danone, Sade', 'Danone, Danette', 'Danone, Danino'],['Danone', 'Danone', 'Danone'],inplace=True)
df1['BRAND'].replace(['Dardanel, Sade', 'Dardenel'],['Dardanel', 'Dardanel'],inplace=True)
df1['BRAND'].replace(['Defans, Fo'],['Defans'],inplace=True)
df1['BRAND'].replace(['Detan, Max', 'Detan, Gece'],['Detan', 'Detan'],inplace=True)
df1['BRAND'].replace(['Dixi, Orkide'],['Dixi'],inplace=True)
df1['BRAND'].replace(['Doğadan, Filiz', 'Doğadan, İçim', 'Doğadan, Sade'],['Doğadan', 'Doğadan', 'Doğadan'],inplace=True)
df1['BRAND'].replace(['Doğanay, Sade'],['Doğanay'],inplace=True)
df1['BRAND'].replace(['Doğuş, Filiz'],['Doğuş'],inplace=True)
df1['BRAND'].replace(['Dolco, Eriş'],['Dolco'],inplace=True)
df1['BRAND'].replace(['Dr. Oetker, Sade', 'Dr Oetker', 'Dr. Oetker, Damla', 'Dr. Oetker, Tat', 'Dr.Oetker', 'Dr.Oetker, Tat', 'Dr.Oetker, Yıldız', 'Dr.Oetker, Sade', 'Dr.oetker'],['Dr. Oetker','Dr. Oetker', 'Dr. Oetker', 'Dr. Oetker', 'Dr. Oetker', 'Dr. Oetker', 'Dr. Oetker', 'Dr. Oetker', 'Dr. Oetker'],inplace=True)
df1['BRAND'].replace(['Dragon Pearl, Eriş'],['Dragon Pearl'],inplace=True)
df1['BRAND'].replace(['Ege, Glutensiz', 'Ege, Tat'],['Ege', 'Ege'],inplace=True)
df1['BRAND'].replace(['Eker, Sade', 'Eker, Tat'],['Eker', 'Eker'],inplace=True)
df1['BRAND'].replace(['Elite, Hero'],['Elite'],inplace=True)
df1['BRAND'].replace(['Ernet, Sade'],['Ernet'],inplace=True)
df1['BRAND'].replace(['Eti, Fo, Sade', 'Eti, Fo', 'Eti, Eti', 'Eti, Tat', 'Eti, Sade', 'Eti, Gold', 'Eti, Boron'],['Eti', 'Eti', 'Eti', 'Eti', 'Eti', 'Eti', 'Eti'],inplace=True)
df1['BRAND'].replace(['Faıry'],['Fairy'],inplace=True)
df1['BRAND'].replace(['Falım, Damla'],['Falım'],inplace=True)
df1['BRAND'].replace(['Filiz, Eriş', 'Filiz, Eriş', 'Filiz, Yıldız'],['Filiz', 'Filiz', 'Filiz'],inplace=True)
df1['BRAND'].replace(['Finish, Max'],['Finish'],inplace=True)
df1['BRAND'].replace(['First, Damla', 'Fırst'],['First', 'First'],inplace=True)
df1['BRAND'].replace(['Fiskokrem'],['Fiskobirlik'],inplace=True)
df1['BRAND'].replace(['Fo, Sade'],['Fo'],inplace=True)
df1['BRAND'].replace(['Frosch, Alo'],['Frosch'],inplace=True)
df1['BRAND'].replace(['G.c., Sade', 'Ricotta'],['G.C.', 'G.C.'],inplace=True)
df1['BRAND'].replace(['Glade, İpek'],['Glade'],inplace=True)
df1['BRAND'].replace(['Grana Padano'],['CİBUS'],inplace=True)
df1['BRAND'].replace(['Green Life, Melis'],['Green Life'],inplace=True)
df1['BRAND'].replace(['Harıbo'],['Haribo'],inplace=True)
df1['BRAND'].replace(['Hasmandıra, Sade'],['Hasmandıra'],inplace=True)
df1['BRAND'].replace(['Hayat, Tas'],['Hayat'],inplace=True)
df1['BRAND'].replace(['Hisar, Damla'],['Hisar'],inplace=True)
df1['BRAND'].replace(['İçim, İçim', 'İçim, Sade'],['İçim', 'İçim'],inplace=True)
df1['BRAND'].replace(['İpek, Tat'],['İpek'],inplace=True)
df1['BRAND'].replace(['Irmak, Kristal'],['Irmak'],inplace=True)
df1['BRAND'].replace(['Jacobs, Gold'],['Jacobs'],inplace=True)
df1['BRAND'].replace(['Jelıbon', 'Kent, Jelibon'],['Jelibon', 'Jelibon'],inplace=True)
df1['BRAND'].replace(['Kenton, Tat', 'Kenton, Sade'],['Kenton', 'Kenton'],inplace=True)
df1['BRAND'].replace(['Kahve Dünyası, İçim', 'Kahve Dünyası, Damla', 'Kd.'],['Kahve Dünyası', 'Kahve Dünyası', 'Kahve Dünyası'],inplace=True)
df1['BRAND'].replace(['Karali, Filiz'],['Karali'],inplace=True)
df1['BRAND'].replace(['Karper, Sade'],['Karper'],inplace=True)
df1['BRAND'].replace(['Kebır Köy'],['Kebir'],inplace=True)
df1['BRAND'].replace(['Kent, Olips', 'Kent, Jelibon'],['Kent', 'Kent'],inplace=True)
df1['BRAND'].replace(['Kenton, Tat', 'Kenton, Sade'],['Kenton', 'Kenton'],inplace=True)
df1['BRAND'].replace(['Kızılay, Sade'],['Kızılay'],inplace=True)
df1['BRAND'].replace(['Knorr, Yayla', 'Knorr, Eriş', 'Knorr, Salat'],['Knorr', 'Knorr', 'Knorr'],inplace=True)
df1['BRAND'].replace(['Kocatepe, Damla'],['Kocatepe'],inplace=True)
df1['BRAND'].replace(['Komili, Ege', 'Komılı'],['Komili', 'Komili'],inplace=True)
df1['BRAND'].replace(['Koska, Sade'],['Koska'],inplace=True)
df1['BRAND'].replace(['Kosla Halı', 'Vanish, Gold', 'Vanısh'],['Vanish', 'Vanish', 'Vanish'],inplace=True)
df1['BRAND'].replace(['Kotex, Gece'],['Kotex'],inplace=True)
df1['BRAND'].replace(['Kühne, Salat', 'Kühne, Ege', 'Kuhne'],['Kühne', 'Kühne', 'Kühne'],inplace=True)
df1['BRAND'].replace(['Kurukahveci Mehmet Efendi, Brazilian'],['Kurukahveci Mehmet Efendi'],inplace=True)
df1['BRAND'].replace(['La Vache, Sade'],['La Vache'],inplace=True)
df1['BRAND'].replace(['Lipton, Sade', 'Lıpton', 'Lipton, Filiz', 'Lipton, Gold'],['Lipton', 'Lipton', 'Lipton', 'Lipton'],inplace=True)
df1['BRAND'].replace(['Lolipop, Sek'],['Lolipop'],inplace=True)
df1['BRAND'].replace(['Makarneks, Eriş'],['Makarneks'],inplace=True)
df1['BRAND'].replace(['Marmarabirlik, Sade', 'Marmarabirlik, Gold'],['Marmarabirlik', 'Marmarabirlik'],inplace=True)
df1['BRAND'].replace(['Melis, Salat', 'Melis, Tat'],['Melis', 'Melis'],inplace=True)
df1['BRAND'].replace(['Mevlana, Sade'],['Mevlana'],inplace=True)
df1['BRAND'].replace(['Mezzetürk, Salat'],['Mezzetürk'],inplace=True)
df1['BRAND'].replace(['Mıxmey', 'Mıxmey, Erikli', 'Mixmey, Erikli'],['Mıxmey', 'Mıxmey', 'Mıxmey'],inplace=True)
df1['BRAND'].replace(['Mr Muscle, Söke', 'Mr. Muscle'],['Mr Muscle', 'Mr Muscle'],inplace=True)
df1['BRAND'].replace(['Nescafe, Gold', 'Nescafé, Gold', 'Nescafé'],['Nescafe', 'Nescafe', 'Nescafe'],inplace=True)
df1['BRAND'].replace(['Nesfit, Sade', 'Nestle, Nesfit'],['Nesfit', 'Nesfit'],inplace=True)
df1['BRAND'].replace(['Nesquık'],['Nesquik'],inplace=True)
df1['BRAND'].replace(['Nestlé', 'Nestle, Gece', 'Nestlé, Gece', 'Nestle, Gold', 'Nestle, Polo', 'Nestlé, Polo', 'Sma 2'],['Nestle','Nestle', 'Nestle', 'Nestle','Nestle','Nestle','Nestle'],inplace=True)
df1['BRAND'].replace(['Nıvea'],['Nivea'],inplace=True)
df1['BRAND'].replace(['Off, Max'],['Off'],inplace=True)
df1['BRAND'].replace(['Omo, Domestos'],['Omo'],inplace=True)
df1['BRAND'].replace(['Oneo, Tat'],['Oneo'],inplace=True)
df1['BRAND'].replace(['Orvıtal Organik'],['Orvital Organik'],inplace=True)
df1['BRAND'].replace(['Pakmaya, Dünya', 'Pakmaya, Sade', 'Pakmaya, Damla'],['Pakmaya', 'Pakmaya', 'Pakmaya'],inplace=True)
df1['BRAND'].replace(['Parex, Jet'],['Parex'],inplace=True)
df1['BRAND'].replace(['Pastavilla, Eriş'],['Pastavilla'],inplace=True)
df1['BRAND'].replace(['Patos Critos'],['Patos'],inplace=True)
df1['BRAND'].replace(['Penguen, Salat'],['Penguen'],inplace=True)
df1['BRAND'].replace(['Pepsi, Max'],['Pepsi'],inplace=True)
df1['BRAND'].replace(['Persıl', 'Persıl, Vernel'],['Persil', 'Persil'],inplace=True)
df1['BRAND'].replace(['Perwoll, İpek'],['Perwoll'],inplace=True)
df1['BRAND'].replace(['Peyman, Çitliyo', 'Peyman, Nutzz'],['Peyman', 'Peyman'],inplace=True)
df1['BRAND'].replace(['Pınar, Sade', 'Pınar, Fo', 'Pınar, Pınar'],['Pınar', 'Pınar', 'Pınar'],inplace=True)
df1['BRAND'].replace(['Pril, Gold', 'Pril, Alo', 'Pril, Ege'],['Pril', 'Pril', 'Pril'],inplace=True)
df1['BRAND'].replace(['Prima, Max', 'Prıma', 'Prıma, Max'],['Prima', 'Prima', 'Prima'],inplace=True)
df1['BRAND'].replace(['Pringles, Sade'],['Pringles'],inplace=True)
df1['BRAND'].replace(['Purelife, Kristal'],['Purelife'],inplace=True)
df1['BRAND'].replace(['Raid, Gece', 'Raid, Max'],['Raid', 'Raid'],inplace=True)
df1['BRAND'].replace(['Reıs', 'Reis, Ege'],['Reis', 'Reis'],inplace=True)
df1['BRAND'].replace(['Rinso, Alo'],['Rinso'],inplace=True)
df1['BRAND'].replace(['SADE ORGANİK ÜR.SAN.VE TİC.A.Ş.'],['SADE'],inplace=True)
df1['BRAND'].replace(['Saffet, Tat', 'Saffet, Kemal'],['Saffet', 'Saffet'],inplace=True)
df1['BRAND'].replace(['S.brıte'],['Scotch Brite'],inplace=True)
df1['BRAND'].replace(['Sek, Sade'],['Sek'],inplace=True)
df1['BRAND'].replace(['Serafressh', 'Serfresh, Sade', 'Serafresh, Sade'],['Serafresh', 'Serafresh', 'Serafresh'],inplace=True)
df1['BRAND'].replace(['Sevgisun, Eriş'],['Sevgisun'],inplace=True)
df1['BRAND'].replace(['Sleepy, Max'],['Sleepy'],inplace=True)
df1['BRAND'].replace(['Sütaş, Sade', 'Sütaş, Tat'],['Sütaş', 'Sütaş'],inplace=True)
df1['BRAND'].replace(['Tahsildaroğlu, Damla'],['Tahsildaroğlu'],inplace=True)
df1['BRAND'].replace(['Takita, Tat, Saka'],['Takita'],inplace=True)
df1['BRAND'].replace(['Tamek, Tat'],['Tamek'],inplace=True)
df1['BRAND'].replace(['Tariş, Ege', 'Tarıs'],['Tariş', 'Tariş'],inplace=True)
df1['BRAND'].replace(['Tas, Eriş'],['Tas'],inplace=True)
df1['BRAND'].replace(['Tat, Salat', 'Tat, Tat'],['Tat', 'Tat'],inplace=True)
df1['BRAND'].replace(['Tchibo, Gold'],['Tchibo'],inplace=True)
df1['BRAND'].replace(['Tire Süt, Koop'],['Tire Süt'],inplace=True)
df1['BRAND'].replace(['Tukas'],['Tukaş'],inplace=True)
df1['BRAND'].replace(['Türkiye Tarım Kredi, Koop'],['Türkiye Tarım Kredi'],inplace=True)
df1['BRAND'].replace(['Tursilmatik', 'Tursılmatık', 'Tursil Matik'],['Tursil', 'Tursil', 'Tursil'],inplace=True)
df1['BRAND'].replace(['Ülker, Altınbaşak', 'Ülker, Damla', 'Ülker, Gold', 'Ülker, Hanımeller','Ülker, Hanımeller, Damla','Ülker, Hanımeller, Tat', 'Ülker, Krispi', 'Ülker, Makarneks, Eriş','Ülker, Oneo','Ülker, Oneo, Frutti','Ülker, Rulokat, Tat','Ülker, Sade', 'Ülker, Tat, Rulokat', 'Tat, Rulokat'],['Ülker', 'Ülker', 'Ülker', 'Ülker', 'Ülker', 'Ülker', 'Ülker', 'Ülker', 'Ülker', 'Ülker', 'Ülker', 'Ülker', 'Ülker','Ülker' ],inplace=True)
df1['BRAND'].replace(['Uludağ, Frutti', 'Uludağ, Frutti, Max', 'Uludağ, Sade'],['Uludağ', 'Uludağ', 'Uludağ'],inplace=True)
df1['BRAND'].replace(['Unabella, Sade'],['Unabella'],inplace=True)
df1['BRAND'].replace(['Uneller, Sade'],['Uneller'],inplace=True)
df1['BRAND'].replace(['Unı Baby'],['Uni'],inplace=True)
df1['BRAND'].replace(['Veg & Bones'],['Veg&bones'],inplace=True)
df1['BRAND'].replace(['Vegan, Sade'],['Vegan'],inplace=True)
df1['BRAND'].replace(['Veggy, Vegan'],['Veggy'],inplace=True)
df1['BRAND'].replace(['Vernel, Max', 'Vernel, Max, Alo', 'Vernel, Max, Perwoll'],['Vernel', 'Vernel', 'Vernel, Perwoll'],inplace=True)
df1['BRAND'].replace(['Viennetta, Gold'],['Viennetta'],inplace=True)
df1['BRAND'].replace(['Vivident, Damla', 'Vıvıdent, Damla', 'Vıvıdent', 'Vıvdent'],['Vivident', 'Vivident', 'Vivident', 'Vivident'],inplace=True)
df1['BRAND'].replace(['Wasa, Sade'],['Wasa'],inplace=True)
df1['BRAND'].replace(['Yayla, Yayla', 'Yayla, Glutensiz'],['Yayla', 'Yayla'],inplace=True)
df1['BRAND'].replace(['Yenigün, Tat'],['Yenigün'],inplace=True)
df1['BRAND'].replace(['Yumos', 'Yumoş, Alo', 'Yumoş, Orkide', 'Yumoş, Lotus'],['Yumoş', 'Yumoş','Yumoş','Yumoş' ],inplace=True)
df1['BRAND'].replace(['Zaika, Eriş'],['Zaika'],inplace=True)
df1['BRAND'].replace(['Züber, Glutensiz'],['Züber'],inplace=True)
df1['BRAND'].replace(['Dr. Beckman'],['Dr.Beckmann'],inplace=True)
df1['BRAND'].replace(['Yudum, Ege'],['Yudum'],inplace=True)
df1['BRAND'].replace(['Mıxmey'],['Mixmey'],inplace=True)
df1['BRAND'].replace(['Sade Organik'],['Sade'],inplace=True)
df1['BRAND'].replace(['ÖNCÜ', 'Öncü', 'Öncü Tat'],['Öncü', 'Öncü','Öncü'],inplace=True)
df1['BRAND'].replace(['Yörükoğlu, Piknik'],['Yörükoğlu'],inplace=True)
df1['BRAND'].replace(['Superfresh, Superton', 'Superton'],['Superfresh', 'Superfresh'],inplace=True)
df1['BRAND'].replace(['Abc', 'Abc, ABC', 'ABC, Gücü', 'Abc, Gücü', 'Abc, Orkide'],['ABC', 'ABC', 'ABC', 'ABC', 'ABC'],inplace=True)
df1['BRAND'].replace(['Clear, Arı'],['Clear'],inplace=True)
df1['BRAND'].replace(['Colgate, Eti', 'Colgate, Gold', 'Colgate, Max', 'Colgate, Fo'],['Colgate', 'Colgate', 'Colgate', 'Colgate'],inplace=True)
df1['BRAND'].replace(['DEL.AXE'],['Axe'],inplace=True)
df1['BRAND'].replace(['Garnier, Garnier', 'Garnier, Alo', 'Garnıer', 'Garnier, Alo', 'Garnıer, Gece'],['Garnier', 'Garnier', 'Garnier', 'Garnier', 'Garnier'],inplace=True)
df1['BRAND'].replace(['Glıss', 'Schwarzkopf, Gliss'],['Gliss', 'Gliss'],inplace=True)
df1['BRAND'].replace(['Head& Shoulders', 'Head & Shoulders, İpek', 'Head Shoulders', 'Head&Shoulders, Arı', 'HEAD & SHOULD', 'Head&Shoulders', 'Head&shoulders', 'Head&Shoulders, İpek'],['Head & Shoulders','Head & Shoulders','Head & Shoulders', 'Head & Shoulders', 'Head & Shoulders', 'Head & Shoulders', 'Head & Shoulders', 'Head & Shoulders'],inplace=True)
df1['BRAND'].replace(['İpek, Max', 'İpek, Orkide', 'Ipek', 'IPEK'],['İpek', 'İpek', 'İpek', 'İpek'],inplace=True)
df1['BRAND'].replace(['Loreal, Gece', 'Loréal', 'Loreal, Alo', 'LOréal'],['Loreal', 'Loreal', 'Loreal', 'Loreal'],inplace=True)
df1['BRAND'].replace(['Neutrogena, Clear', 'Neutrogena, Clear, Arı'],['Neutrogena', 'Neutrogena'],inplace=True)
df1['BRAND'].replace(['Nivea, Alo', 'Nivea, Clear', 'Nivea, Fo', 'Nivea, Gece', 'Nivea, İpek', 'Nivea, Kdn', 'Nıvea, Clear'],['Nivea', 'Nivea', 'Nivea', 'Nivea', 'Nivea', 'Nivea', 'Nivea'],inplace=True)
df1['BRAND'].replace(['Old Spice, Clear'],['Old Spice'],inplace=True)
df1['BRAND'].replace(['Öncü, Tat', 'ONCU'],['Öncü', 'Öncü'],inplace=True)
df1['BRAND'].replace(['PANTENE', 'Pantene, İpek', 'Pantene, Arı'],['Pantene', 'Pantene', 'Pantene'],inplace=True)
df1['BRAND'].replace(['Peros, Orkide'],['Peros'],inplace=True)
df1['BRAND'].replace(['Sebamed, Clear'],['Sebamed'],inplace=True)
df1['BRAND'].replace(['Vanish, Vanish'],['Vanish'],inplace=True)
df1['BRAND'].replace(['Duru, Orkide', 'Duru, Alo', 'Duru, Lotus'],['Duru', 'Duru', 'Duru'],inplace=True)
df1['BRAND'].replace(['Rexona, Alo'],['Rexona'],inplace=True)
df1['BRAND'].replace(['Rexona, Alo'],['Rexona'],inplace=True)



df1['BRAND'][df1['MEHSUL LİNKI'] == "https://marketkarsilastir.com/fiyat/8681349442106-probiyotik-meyveli-yogurt-320-g"] = "FROPIE"
df1['BRAND'][df1['MEHSUL LİNKI'] == "https://marketkarsilastir.com/fiyat/8697447040823-turk-kahvesi-teneke-orta"] = "Kahve Dünyası"
df1['BRAND'][df1['MEHSUL LİNKI'] == "https://marketkarsilastir.com/fiyat/8697447040823-turk-kahvesi-teneke-orta"] = "Kahve Dünyası"
df1['BRAND'][df1['MEHSUL LİNKI'] == "https://marketkarsilastir.com/fiyat/8698532591596-ped-tasima-kabi"] = "DIGER"


df1['BRAND'] = df1['BRAND'].str.upper()

In [10]:
# Rename the column "SKU QIYMET" to "SKU QIYMET TL"
df1.rename(columns={'SKU QIYMET': 'SKU QIYMET TL'}, inplace=True)

# Define the exchange rate from TL to AZN
exchange_rate = 0.074311145

# Create a new column "Exchange Rate" with a constant value of the exchange rate
df1['Exchange Rate'] = exchange_rate

# Create the new column "SKU QIYMET AZN" by multiplying "SKU QIYMET TL" with the exchange rate
df1['SKU QIYMET AZN'] = df1['SKU QIYMET TL'] * exchange_rate

# Round the values in "SKU QIYMET AZN" to two decimal places
df1['SKU QIYMET AZN'] = df1['SKU QIYMET AZN'].round(2)

In [11]:
df1

,TARIX,KATEQORIYA,MEHSUL AD,MEHSUL BARKOD,NET MIKTAR,MENSEİ,İÇİNDEKİLER,ALERJEN UYARISI,SAKLAMA KOŞULLARI,GIDA İŞLETMECİSİ / ÜRETİCİ / İTHALATÇI / DAĞITICI,MARKET ADI,SKU AD,SKU QIYMET TL,SON QIYMET DEYİŞİM TARİXİ,MEHSUL SEKİL LİNKİ,MEHSUL LİNKI,BRAND,Exchange Rate,SKU QIYMET AZN
0,2023-06-16 10:10:08.305330,SUT-KAHVALTILIK,Nutella 400 G,80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,Bizim Toptan,Nutella 400 G,53.90,2023-05-24 14:33:00,https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g,NUTELLA,0.074311,4.01
1,2023-06-16 10:10:08.327225,SUT-KAHVALTILIK,Nutella 400 G,80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,Özdilek Hipermarketleri,Nutella 400 G,54.95,2023-05-09 01:26:00,https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g,NUTELLA,0.074311,4.08
2,2023-06-16 10:10:08.332513,SUT-KAHVALTILIK,Nutella 400 G,80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,CarrefourSA,Nutella 400 G,55.00,2023-05-03 01:29:00,https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g,NUTELLA,0.074311,4.09
3,2023-06-16 10:10:08.336578,SUT-KAHVALTILIK,Nutella 400 G,80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,Macrocenter,Nutella 400 G,55.00,2023-05-05 04:34:00,https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g,NUTELLA,0.074311,4.09
4,2023-06-16 10:10:08.341713,SUT-KAHVALTILIK,Nutella 400 G,80135876,400,TÜRKİYE,"şeker,\nbitkisel yağ (palm),\nfındık (%13),\nyağsız süt tozu (%8.7),\nyağı azaltılmış kakao tozu (%7.4),\nemülgatör: lesitin (soya),\naroma verici(vanilin).","fındık ,\nyağsız süt tozu ,\nemülgatör: lesitin (soya),",Serin ve Kuru ortamda muhafaza ediniz,FERRERO TÜRKİYE ÇİKOLATA VE TARIM ÜRÜNLERİ SAN.VE,İstegelsin,Nutella 400 G,55.00,2023-05-08 11:27:00,https://marketkarsilastir.com/urunler/80135876.jpg,https://marketkarsilastir.com/fiyat/80135876-nutella-400-g,NUTELLA,0.074311,4.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26966,2023-06-16 15:36:18.527558,DETERJAN-TEMIZLIK,Pantene Saç Kremi Yag Terapisi 275ML,8006540440728,<NA>,N/A,N/A,N/A,N/A,N/A,Metro Toptancı Market,Pantene Saç Kremi Yag Terapisi 275ML,45.90,2023-01-12 09:37:00,https://marketkarsilastir.com/urunler/8006540440728.jpg,https://marketkarsilastir.com/fiyat/8006540440728-pantene-sac-kremi-yag-terapisi-275ml,PANTENE,0.074311,3.41
26967,2023-06-16 15:36:20.093201,DETERJAN-TEMIZLIK,Pantene Şampuan Yağ Terapisi 350ML,8006540476956,<NA>,N/A,N/A,N/A,N/A,N/A,Bizim Toptan,Pantene Şampuan Yağ Terapisi 350ML,45.95,2023-03-02 09:27:00,https://marketkarsilastir.com/urunler/8006540476956.jpg,https://marketkarsilastir.com/fiyat/8006540476956-pantene-sampuan-yag-terapisi-350ml,PANTENE,0.074311,3.41
26968,2023-06-16 15:36:20.668143,DETERJAN-T

In [12]:
def get_vat_coefficient():
    VAT = 1.18  # Update this value as per your requirements in the future
    return VAT

# get the VAT coefficient
VAT_coefficient = get_vat_coefficient()

# calculate SKU QIYMET AZN and SKU QIYMET TL minus the VAT and round it to 2 decimal points
df1['SKU QIYMET AZN minus VAT 18%'] = (df1['SKU QIYMET AZN'] / VAT_coefficient).round(2)
df1['SKU QIYMET TL minus VAT 18%'] = (df1['SKU QIYMET TL'] / VAT_coefficient).round(2)


In [13]:
def get_coefficient(transport=6, customs_procedures=15.5, other_costs=3, vat=18, margin=0):
    """
    Calculate the coefficient for landed cost.
    """
    # convert percentages to decimals
    transport = 1 + (transport / 100)
    customs_procedures = 1 + (customs_procedures / 100)
    other_costs = 1 + (other_costs / 100)
    vat = 1 + (vat / 100)
    margin = 1 + (margin / 100)

    # calculate coefficient
    coefficient = transport * customs_procedures * other_costs * vat * margin

    return coefficient

# You can use the get_coefficient function to get the coefficient a
coefficient = get_coefficient()

#Then, create a new column named 'Landed Cost AZN' and round it to 2 decimal points
df1['Landed Cost AZN 1,49'] = (df1['SKU QIYMET AZN minus VAT 18%'] * coefficient).round(2)


In [14]:

# Calculate the minimum price for each product and round it to 2 decimal points
min_prices = df1.groupby('MEHSUL BARKOD')['SKU QIYMET AZN minus VAT 18%'].transform('min').round(2)

# Calculate the average price for each product and round it to 2 decimal points
avg_prices = df1.groupby('MEHSUL BARKOD')['SKU QIYMET AZN minus VAT 18%'].transform('mean').round(2)

# Calculate the number of data points for each product
market_count = df1.groupby('MEHSUL BARKOD')['SKU QIYMET AZN minus VAT 18%'].transform('count')

# Create conditional columns for minimum price, average price and market count
df1['Minimum Price'] = np.where(df1['SKU QIYMET AZN minus VAT 18%'].round(2) == min_prices, min_prices, np.nan)
df1['Average Price'] = np.where(df1['Minimum Price'].notna(), avg_prices, np.nan)
df1['Market Sayi'] = np.where(df1['Minimum Price'].notna(), market_count, np.nan)

# Calculate how much lower the minimum price is from the average in percentage and round it to 2 decimal points
df1['Lower than Average (%)'] = np.where(df1['Minimum Price'].notna(), ((df1['Average Price'] - df1['Minimum Price']) / df1['Average Price']), np.nan).round(2)


In [ ]:
# import re
# import pandas as pd
# import numpy as np

# # Assuming df1['SKU AD'] contains the product strings
# def extract_weight_vol(product):
#     product = product.lower().replace(",", ".").replace("lt", "1000 ml").replace("kg", "1000 g")

#     if "x" in product:
#         multiplier_weight = re.findall(r'(\d+)\s*x\s*(\d+)', product)
#         if multiplier_weight:  # if multiplier and weight are found
#             multiplier, weight = multiplier_weight[0]
#             return int(multiplier) * float(weight)
#     elif re.search(r'\d+\s*(ml|g)', product):
#         weight = re.findall(r'(\d+\.\d+|\d+)\s*(ml|g)', product)
#         if weight:  # if weight is found
#             # if unit is 'g' or 'ml', return the value as is
#             if weight[0][1] in ['g', 'ml']:
#                 return float(weight[0][0])
#             else:
#                 return float(weight[0][0]) * 1000
#     elif re.search(r'\d+\s*(l)', product):
#         volume = re.findall(r'(\d+\.\d+|\d+)\s*(l)', product)
#         if volume:  # if volume is found
#             return float(volume[0][0]) * 1000
#     else:
#         return 'n/a'

# # Copy 'NET MIKTAR' values to 'weight_vol' column
# df1['weight_vol'] = df1['NET MIKTAR']

# # Apply extract_weight_vol only to rows where 'NET MIKTAR' is 'n/a', NaN, None, or empty string
# df1.loc[df1['NET MIKTAR'].isin(['n/a', np.nan, None, '']), 'weight_vol'] = df1.loc[df1['NET MIKTAR'].isin(['n/a', np.nan, None, '']), 'SKU AD'].apply(extract_weight_vol)


In [15]:
# Initial number of rows
initial_rows = df1.shape[0]
print(f"Initial number of rows: {initial_rows}")

# Remove duplicates
df1 = df1.drop_duplicates()

# Final number of rows
final_rows = df1.shape[0]
print(f"Final number of rows: {final_rows}")

# Display how many duplicates were removed
print(f"Number of duplicates removed: {initial_rows - final_rows}")

# Reset the index to get the row numbers
df1 = df1.reset_index(drop=True)

Initial number of rows: 26971
Final number of rows: 26971
Number of duplicates removed: 0


In [16]:
# Reorder the columns in df1
df1_columns = ['TARIX', 'KATEQORIYA', 'MENSEİ', 'GIDA İŞLETMECİSİ / ÜRETİCİ / İTHALATÇI / DAĞITICI', 
               'BRAND', 'MEHSUL BARKOD', 'SKU AD', 'NET MIKTAR','MARKET ADI', 'SKU QIYMET TL',
               'SKU QIYMET TL minus VAT 18%', 'Exchange Rate', 'SKU QIYMET AZN',
               'SKU QIYMET AZN minus VAT 18%', 'Minimum Price', 'Average Price',
               'Lower than Average (%)', 'Market Sayi', 'Landed Cost AZN 1,49', 'SON QIYMET DEYİŞİM TARİXİ',
               'MEHSUL SEKİL LİNKİ', 'MEHSUL LİNKI']

df1 = df1.reindex(columns=df1_columns)

# Generate the Excel file name
TodaysDate = time.strftime("%d-%m-%Y")
excelfilename = '/Users/FaiqRasulov/Desktop/Qiymet_list/TOPDANLAR/Market_Karshilashtir/market_karshilashtir ' + TodaysDate + ".xlsx"

# Create an Excel writer
writer = pd.ExcelWriter(excelfilename, engine='xlsxwriter')

# Save df1 to a sheet
df1.to_excel(writer, sheet_name='df1', index=False, header=True)

# Close the writer and save the Excel file
writer.save()

# Multithreaded code

In [ ]:
# import os
# import json
# import concurrent.futures
# import psutil
# import pandas as pd
# from datetime import datetime
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import TimeoutException
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup

# # Define categories and number of pages for each category
# categories = [
#     'sut-kahvaltilik',
#     'temel-gida',
#     'unlu-mamul-pasta',
#     'icecek',
#     'atistirmalik',
#     'meze-hazir-yemek-donuk',
#     'bebek-anne',
#     'deterjan-temizlik'
# ]

# pages = [
#     25,  # Number of pages for 'sut-kahvaltilik'
#     29,  # Number of pages for 'temel-gida'
#     9,   # Number of pages for 'unlu-mamul-pasta'
#     22,  # Number of pages for 'icecek'
#     16,  # Number of pages for 'atistirmalik'
#     2,   # Number of pages for 'meze-hazir-yemek-donuk'
#     13,  # Number of pages for 'bebek-anne'
#     29   # Number of pages for 'deterjan-temizlik'
# ]

# # Define additional URLs for each category
# extra_pages = {
#     'bebek-anne': [
#         "https://marketkarsilastir.com/ara/kotex"
#     ], 
#     'temel-gida': [
#         "https://marketkarsilastir.com/ara/oncu"
#     ],
#     'sut-kahvaltilik': [
#         "https://marketkarsilastir.com/ara/yorukoglu"
#     ],
#     'atistirmalik': [
#         "https://marketkarsilastir.com/ara/lays"
#     ],
#     'deterjan-temizlik': [
#         "https://marketkarsilastir.com/ara/abc",
#         "https://marketkarsilastir.com/ara/vanish",
#         "https://marketkarsilastir.com/ara/ariel",
#         "https://marketkarsilastir.com/ara/bref",
#         "https://marketkarsilastir.com/ara/clin",
#         "https://marketkarsilastir.com/ara/clear",
#         "https://marketkarsilastir.com/ara/head-and-shoulders",
#         "https://marketkarsilastir.com/ara/gliss",
#         "https://marketkarsilastir.com/ara/garnier",
#         "https://marketkarsilastir.com/ara/loreal",
#         "https://marketkarsilastir.com/ara/protex",
#         "https://marketkarsilastir.com/ara/duru",
#         "https://marketkarsilastir.com/ara/ipek",
#         "https://marketkarsilastir.com/ara/nivea",
#         "https://marketkarsilastir.com/ara/peros",
#         "https://marketkarsilastir.com/ara/fairy",
#         "https://marketkarsilastir.com/ara/calgon"
#     ]
# }

# # Initialize the list to hold all data
# df_list = []

# # Your data extraction function
# def extract_data(arg):
#     category, url, page = arg

#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     driver.get(url)

#     soup = BeautifulSoup(driver.page_source, 'lxml')

#     # Get all product links
#     product_links = [a['href'] for a in soup.select('a.product-img')]

#     # Loop over each product link
#     for link in product_links:
#         product_url = 'https://marketkarsilastir.com' + ('' if link.startswith('/') else '/') + link
#         driver.get(product_url)

#         product_soup = BeautifulSoup(driver.page_source, 'lxml')

#         # Extract details
#         try:
#             product_name_tag = product_soup.select_one('h1.text-center')
#             product_name = product_name_tag.text.strip() if product_name_tag else 'No name available'

#             product_barcode_tag = product_soup.select_one('h2.fa-1x.text-center')
#             product_barcode = product_barcode_tag.text.strip() if product_barcode_tag else 'No barcode available'

#             product_picture_link_tag = product_soup.select_one('img.img-responsive')
#             product_picture_link = product_picture_link_tag['src'] if product_picture_link_tag else 'No image available'

#             # Extract product info
#             product_info_section = product_soup.find('div', class_='barkodResim')
#             product_info = product_info_section.text.strip() if product_info_section else 'No product info available'

#             # Extract specific info fields
#             net_miktar = 'N/A'
#             mensi = 'N/A'
#             icindekiler = 'N/A'
#             alerjen_uyarisi = 'N/A'
#             saklama_kosullari = 'N/A'
#             gida_isletmecisi = 'N/A'

#             info_items = product_info_section.find_all('p')
#             for item in info_items:
#                 strong_tag = item.find('strong')
#                 if strong_tag:
#                     strong_text = strong_tag.text.strip()
#                     if strong_text == 'Net Miktar (g/ml)':
#                         net_miktar = item.contents[-1].strip()
#                     elif strong_text == 'Menşei':
#                         mensi = item.contents[-1].strip()
#                     elif strong_text == 'İçindekiler':
#                         icindekiler = item.contents[-1].strip()
#                     elif strong_text == 'Alerjen Uyarısı':
#                         alerjen_uyarisi = item.contents[-1].strip()
#                     elif strong_text == 'Saklama Koşulları':
#                         saklama_kosullari = item.contents[-1].strip()
#                     elif strong_text == 'Gıda İşletmecisi / Üretici / İthalatçı / Dağıtıcı':
#                         gida_isletmecisi = item.contents[-1].strip()

#             # Extract product price info
#             product_price_info_section = product_soup.find('div', class_='col-xl-8')
#             product_price_info_rows = product_price_info_section.select('tbody tr')

#             for row in product_price_info_rows:
#                 market_name = row.select_one('td a img')['title']
#                 sku_name = row.select_one('td:nth-child(3) a').text.strip()
#                 sku_price = row.select_one('td:nth-child(4)').text.strip()
#                 latest_price_change = row.select_one('td:nth-child(3) small').text.strip()

#                 # Save details to dataframe
#                 df_list.append({
#                     'TARIX': datetime.today(),
#                     'KATEQORIYA': category,
#                     'MEHSUL AD': product_name,
#                     'MEHSUL BARKOD': product_barcode,
#                     'NET MIKTAR': net_miktar,
#                     'MENSEİ': mensi,
#                     'İÇİNDEKİLER': icindekiler,
#                     'ALERJEN UYARISI': alerjen_uyarisi,
#                     'SAKLAMA KOŞULLARI': saklama_kosullari,
#                     'GIDA İŞLETMECİSİ / ÜRETİCİ / İTHALATÇI / DAĞITICI': gida_isletmecisi,
#                     'MARKET ADI': market_name,
#                     'SKU AD': sku_name,
#                     'SKU QIYMET': sku_price,
#                     'SON QIYMET DEYİŞİM TARİXİ': latest_price_change,
#                     'MEHSUL SEKİL LİNKİ': product_picture_link,
#                     'MEHSUL LİNKI': product_url
#                 })

#         except Exception as e:
#             print(f"Error extracting data from {product_url}: {str(e)}")

#         finally:
#             driver.quit()

# # Replace your original loop with this
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # Flatten your URLs into a list of arguments for extract_data
#     args = [(category, f"https://marketkarsilastir.com/urunkat/{category}/?sayfa={page}", page)
#             for category, page_count in zip(categories, pages)
#             for page in range(1, page_count + 1)]
#     # Then add the extra URLs
#     args.extend((category, url, None) for category, urls in extra_pages.items() for url in urls)
#     # Start the load operations and mark each future with its URL
#     future_to_url = {executor.submit(extract_data, arg): arg for arg in args}
#     for future in concurrent.futures.as_completed(future_to_url):
#         url = future_to_url[future]
#         try:
#             data = future.result()
#         except Exception as exc:
#             print(f'{url} generated an exception: {exc}')
#         else:
#             print(f'{url} page is complete')

# # Print CPU usage statistics
# cpu_usage = psutil.cpu_percent(interval=1)
# print(f'CPU usage: {cpu_usage}%')

# # Get the current date as a string
# date_string = datetime.now().strftime('%Y-%m-%d')

# # Create the filename
# filename = '/Users/FaiqRasulov/Desktop/Qiymet_list/TOPDANLAR/Market_Karshilashtir/marketkarsilastir_{date_string}.csv'

# # Create DataFrame from the list of dictionaries
# df = pd.DataFrame(df_list)

# # Rearrange the columns
# df = df[['TARIX', 'KATEQORIYA', 'MEHSUL AD', 'MEHSUL BARKOD', 'NET MIKTAR', 'MENSEİ', 'İÇİNDEKİLER',
#          'ALERJEN UYARISI', 'SAKLAMA KOŞULLARI', 'GIDA İŞLETMECİSİ / ÜRETİCİ / İTHALATÇI / DAĞITICI',
#          'MARKET ADI', 'SKU AD', 'SKU QIYMET', 'SON QIYMET DEYİŞİM TARİXİ', 'MEHSUL SEKİL LİNKİ', 'MEHSUL LİNKI']]

# # Save to CSV
# df.to_csv(filename, index=False, sep=';', encoding='utf-16')


# Original Code

In [ ]:
# # Initialize webdriver
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# # Define categories and number of pages for each category
# categories = [
#     'sut-kahvaltilik',
#     'temel-gida',
#     'unlu-mamul-pasta',
#     'icecek',
#     'atistirmalik',
#     'meze-hazir-yemek-donuk',
#     'bebek-anne',
#     'deterjan-temizlik'
# ]

# pages = [
#     25,  # Number of pages for 'sut-kahvaltilik'
#     29,  # Number of pages for 'temel-gida'
#     9,   # Number of pages for 'unlu-mamul-pasta'
#     22,  # Number of pages for 'icecek'
#     16,  # Number of pages for 'atistirmalik'
#     2,   # Number of pages for 'meze-hazir-yemek-donuk'
#     13,  # Number of pages for 'bebek-anne'
#     29   # Number of pages for 'deterjan-temizlik'
# ]


# # Define additional URLs for each category
# extra_pages = {
#     'bebek-anne': [
#         "https://marketkarsilastir.com/ara/kotex"
#     ], 
#     'temel-gida': [
#         "https://marketkarsilastir.com/ara/oncu"
#     ],
#     'sut-kahvaltilik': [
#         "https://marketkarsilastir.com/ara/yorukoglu"
#     ],
#     'atistirmalik': [
#         "https://marketkarsilastir.com/ara/lays"
#     ],
#     'deterjan-temizlik': [
#         "https://marketkarsilastir.com/ara/abc",
#         "https://marketkarsilastir.com/ara/vanish",
#         "https://marketkarsilastir.com/ara/ariel",
#         "https://marketkarsilastir.com/ara/bref",
#         "https://marketkarsilastir.com/ara/clin",
#         "https://marketkarsilastir.com/ara/clear",
#         "https://marketkarsilastir.com/ara/head-and-shoulders",
#         "https://marketkarsilastir.com/ara/gliss",
#         "https://marketkarsilastir.com/ara/garnier",
#         "https://marketkarsilastir.com/ara/loreal",
#         "https://marketkarsilastir.com/ara/protex",
#         "https://marketkarsilastir.com/ara/duru",
#         "https://marketkarsilastir.com/ara/ipek",
#         "https://marketkarsilastir.com/ara/nivea",
#         "https://marketkarsilastir.com/ara/peros",
#         "https://marketkarsilastir.com/ara/rexona",
#         "https://marketkarsilastir.com/ara/selpak",
#         "https://marketkarsilastir.com/ara/solo",
#         "https://marketkarsilastir.com/ara/papia",
#         "https://marketkarsilastir.com/ara/calgon",
#         "https://marketkarsilastir.com/ara/colgate",
#         "https://marketkarsilastir.com/ara/pantene"
#     ]
#     # Add additional categories and URLs as needed...
# }


# df_list = []

# # Your data extraction function
# def extract_data(category, url):
#     driver.get(url)
#     time.sleep(2)  # Wait for the page to load

#     soup = BeautifulSoup(driver.page_source, 'lxml')

#     # Get all product links
#     product_links = [a['href'] for a in soup.select('a.product-img')]

#     # Loop over each product link
#     for link in product_links:
#         product_url = 'https://marketkarsilastir.com' + ('' if link.startswith('/') else '/') + link
#         driver.get(product_url)

#         WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1.text-center')))

#         product_soup = BeautifulSoup(driver.page_source, 'lxml')

#         # Extract details
#         try:
#             product_name_tag = product_soup.select_one('h1.text-center')
#             product_name = product_name_tag.text.strip() if product_name_tag else 'No name available'

#             product_barcode_tag = product_soup.select_one('h2.fa-1x.text-center')
#             product_barcode = product_barcode_tag.text.strip() if product_barcode_tag else 'No barcode available'

#             product_picture_link_tag = product_soup.select_one('img.img-responsive')
#             product_picture_link = product_picture_link_tag['src'] if product_picture_link_tag else 'No image available'

#             # Extract product info
#             product_info_section = product_soup.find('div', class_='barkodResim')
#             product_info = product_info_section.text.strip() if product_info_section else 'No product info available'

#             # Extract specific info fields
#             net_miktar = 'N/A'
#             mensi = 'N/A'
#             icindekiler = 'N/A'
#             alerjen_uyarisi = 'N/A'
#             saklama_kosullari = 'N/A'
#             gida_isletmecisi = 'N/A'

#             info_items = product_info_section.find_all('p')
#             for item in info_items:
#                 strong_tag = item.find('strong')
#                 if strong_tag:
#                     strong_text = strong_tag.text.strip()
#                     if strong_text == 'Net Miktar (g/ml)':
#                         net_miktar = item.contents[-1].strip()
#                     elif strong_text == 'Menşei':
#                         mensi = item.contents[-1].strip()
#                     elif strong_text == 'İçindekiler':
#                         icindekiler = item.contents[-1].strip()
#                     elif strong_text == 'Alerjen Uyarısı':
#                         alerjen_uyarisi = item.contents[-1].strip()
#                     elif strong_text == 'Saklama Koşulları':
#                         saklama_kosullari = item.contents[-1].strip()
#                     elif strong_text == 'Gıda İşletmecisi / Üretici / İthalatçı / Dağıtıcı':
#                         gida_isletmecisi = item.contents[-1].strip()

#             # Extract product price info
#             product_price_info_section = product_soup.find('div', class_='col-xl-8')
#             product_price_info_rows = product_price_info_section.select('tbody tr')

#             for row in product_price_info_rows:
#                 market_name = row.select_one('td a img')['title']
#                 sku_name = row.select_one('td:nth-child(3) a').text.strip()
#                 sku_price = row.select_one('td:nth-child(4)').text.strip()
#                 latest_price_change = row.select_one('td:nth-child(3) small').text.strip()

#                 # Save details to dataframe
#                 df_list.append({
#                     'TARIX': datetime.today(),
#                     'KATEQORIYA': category,
#                     'MEHSUL AD': product_name,
#                     'MEHSUL BARKOD': product_barcode,
#                     'NET MIKTAR': net_miktar,
#                     'MENSEİ': mensi,
#                     'İÇİNDEKİLER': icindekiler,
#                     'ALERJEN UYARISI': alerjen_uyarisi,
#                     'SAKLAMA KOŞULLARI': saklama_kosullari,
#                     'GIDA İŞLETMECİSİ / ÜRETİCİ / İTHALATÇI / DAĞITICI': gida_isletmecisi,
#                     'MARKET ADI': market_name,
#                     'SKU AD': sku_name,
#                     'SKU QIYMET': sku_price,
#                     'SON QIYMET DEYİŞİM TARİXİ': latest_price_change,
#                     'MEHSUL SEKİL LİNKİ': product_picture_link,
#                     'MEHSUL LİNKI': product_url
#                 })

#         except Exception as e:
#             print(f"Error extracting data from {product_url}: {str(e)}")

# # Start extracting data from the original URLs
# for category, page_count in zip(categories, pages):
#     for page in range(1, page_count + 1):
#         url = f"https://marketkarsilastir.com/urunkat/{category}/?sayfa={page}"
#         extract_data(category, url)

# # Now extract data from the additional URLs
# for category, urls in extra_pages.items():
#     for url in urls:
#         extract_data(category, url)

# # Close the driver
# driver.quit()

# # Create DataFrame from the list of dictionaries
# df = pd.DataFrame(df_list)

# # Rearrange the columns
# df = df[['TARIX', 'KATEQORIYA', 'MEHSUL AD', 'MEHSUL BARKOD', 'NET MIKTAR', 'MENSEİ', 'İÇİNDEKİLER',
#          'ALERJEN UYARISI', 'SAKLAMA KOŞULLARI', 'GIDA İŞLETMECİSİ / ÜRETİCİ / İTHALATÇI / DAĞITICI',
#          'MARKET ADI', 'SKU AD', 'SKU QIYMET', 'SON QIYMET DEYİŞİM TARİXİ', 'MEHSUL SEKİL LİNKİ', 'MEHSUL LİNKI']]

# # Print the DataFrame
# print(df)